In [1]:
import pandas as pd
import multiprocessing
from multiprocessing import Pool
import numpy as np


In [2]:
# Load the data

DATABASE_PATH = 'data/model_fraud_resume.csv'
df = pd.read_csv(DATABASE_PATH)

In [3]:
# Copy of the original dataframe to compare the results

df_original = df.copy()

In [4]:
#ENRICHMENT
# Create a new column with the difference between the initial balance and the amount of the transaction

df['deltaOrigen'] = (df['saldoInicialOrigen'] -df['monto']).abs()

In [5]:
# ALGORITHM FOR RETIROS
# first rule: if the transaction is a retiro, check if there is a transfer with the same amount in the last 3 time units

def check_for_retiro(row, df:pd.DataFrame):
    tiempo = row['unidadTiempo'] 
    tipo_transaccion=row['tipoTransaccion']
    monto_transaccion = row['monto']
    saldo_inicial_origen = row['saldoInicialOrigen']
    
    if tipo_transaccion == 'Retiro':    
        df = df[df['tipoTransaccion'] == 'Transferencia']
        df = df[(df['unidadTiempo'] == tiempo) | (df['unidadTiempo'] == tiempo-1) | (df['unidadTiempo'] == tiempo-2)]
        df = df[df['monto'] == monto_transaccion]
        
        if len(df) > 0:
            return 1
        
    return 0

        

In [6]:
# ALGORITHM FOR TRANSFERENCIAS
# first rule: if the transaction is a transferencia, check if the deltaOrigen is 0 

def check_for_transferencia(row):
    tipo_transaccion=row['tipoTransaccion']
    delta_origen = row['deltaOrigen']
    saldo_inicial_destinatario = row['saldoInicialDestinatario']
    saldo_final_destinatario = row['saldoFinalDestinatario']
    saldo_final_origen = row['saldoFinalOrigen']
    
    if tipo_transaccion == 'Transferencia':
        if delta_origen == 0:
            #if saldo_inicial_destinatario == 0 and saldo_final_destinatario == 0 and saldo_final_origen == 0:
                return 1
    
    return 0

In [7]:
# NEW COLUMN WITH THE RESULTS OF THE ALGORITHMS

df['marca_fraude_proyectada'] = 0

In [8]:
# Data frame just for retiro and transferencia
df_retiro = df[df['tipoTransaccion'] == 'Retiro']
df_transferencia = df[df['tipoTransaccion'] == 'Transferencia']

In [9]:
def apply_check_for_transferencia(df_transferencia):
    result = []
    for index, row in df_transferencia.iterrows():
        marca = check_for_transferencia(row)
        if marca == 1:
            result.append(index)
    return result
        


In [11]:
print(apply_check_for_transferencia(df_transferencia))

df_transferencia_split = [df_transferencia.iloc[i:i+1000] for i in range(0, len(df_transferencia), 1000)]


with Pool(4) as p:
    result = p.map(apply_check_for_transferencia, df_transferencia_split)



[2, 251, 680, 969, 1115, 1869, 2301, 3059, 3162, 3271, 3683, 4103, 4260, 4442, 4667, 4693, 4775, 4857, 5123, 5466, 5558, 5746, 5852, 5994, 6583, 6702, 6706, 6897, 7002, 7029, 7154, 9284]
